In [11]:
# https://www.youtube.com/watch?v=SEDwevQ0wic

#Giovanni Andretta Carbonero

import cv2
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
import time
%matplotlib inline

Usuario = []
Senha = []

def faceDetection(test_img):
    gray_img=cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
    face_haar_cascade=cv2.CascadeClassifier('DeteccaoFace/haarcascades/haarcascade_frontalface_default.xml')
    faces=face_haar_cascade.detectMultiScale(gray_img,scaleFactor=1.3,minNeighbors=5)
    return faces,gray_img

def train_classifier(faces,face_ID):
    face_recognizer=cv2.face.LBPHFaceRecognizer_create()
    face_recognizer.train(faces,np.array(faceID))
    return face_recognizer

def draw_rect(test_img,face):
    (x,y,w,h)=face
    cv2.rectangle(test_img,(x,y),(x+w,y+h),(0,255,0),thickness=3)
    
def put_text(test_img,text,x,y):
    cv2.putText(test_img,text,(x,y),cv2.FONT_HERSHEY_DUPLEX,1,(0,0,255),1)
    
def labels_for_training_data(directory):
    faces=[]
    faceID=[]
    
    for path,subdirnames,filenames in os.walk(directory):
        for filename in filenames:
            if filename.startswith("."):
                print("Skypping system files")
                continue
            id=os.path.basename(path)
            img_path=os.path.join(path,filename)
            print("img_path",img_path)
            print("id:",id)
            test_img=cv2.imread(img_path)
            if test_img is None:
                print("Image not loaded properly")
                
                continue
            faces_rect,gray_img=faceDetection(test_img)
            if len(faces_rect)!=1:
                continue
                
            (x,y,w,h)=faces_rect[0]
            rol_gray=gray_img[y:y+w,x:x+h]
            faces.append(rol_gray)
            faceID.append(int(id))
        return faces,faceID

def validacao():
    global faceID, confidence
    confidence=0
    
    test_img=cv2.imread('Temp/temp.jpg')
    faces_detected,gray_img=faceDetection(test_img)
    print("face detected",faces_detected)

    faces,faceID=labels_for_training_data('Face/0')
    face_recognizer=train_classifier(faces,faceID)
    face_recognizer.save('trainingdata.yml')
    face_recognizer=cv2.face.LBPHFaceRecognizer_create()
    face_recognizer.read('trainingdata.yml')
    
    name={0:Usuario[0]}

    for face in faces_detected:
        (x,y,w,h)=face
        rol_gray=gray_img[y:y+h,x:x+h]
        label,confidence=face_recognizer.predict(rol_gray)
        print("Confidence",confidence)
        print("Label",label)
        draw_rect(test_img,face)
        predict_name=name[label]
        
        #Verifica se é ou não a pessoa e coloca o nome
        if confidence>68:
            continue
        put_text(test_img,predict_name,x,y)

    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=1)
        pass
    
    resized_img=cv2.resize(test_img,(1000,700))
    cv2.imshow("Face Detection",resized_img)
    
    #Temporizador da janela de validacao
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    #Verifica se é ou não a pessoa autorizada
    if confidence>68 or confidence==0:
        opt = str(input("\n\n\nRosto não identificado, deseja fazer a validação por usuario e senha? ('S' para sim): "))
        opt = opt.upper()
        opt = opt[0]
        
        if opt == "S":
            login()
        else:
            sys.exit()
    else:
        print("\n\n\t\tBem vindo sr(a). "+Usuario[0]+"\n")
        
def login():
    opt = 3
    
    while True:
        print("\n\n\t\tLogin")
        usuario = str(input("\nUsuario: "))
        senha = str(input("Senha: "))

        if usuario in Usuario[:]:
            if senha in Senha[:]:
                print("\n\n\t\tBem vindo sr(a). "+Usuario[0]+"\n")
                break
            else:
                opt-=1
                print("Usuario ou senha incorretos!\n\nTente novamente... tentativas restantes: " + str(opt))
        else:
            opt-=1
            print("Usuario ou senha incorretos!\n\nTente novamente... tentativas restantes: " + str(opt))

        if opt == 0:
            print("\n\nFechando programa.")
            sys.exit()

def temp():
    camera_port = 0
    camera = cv2.VideoCapture(camera_port)

    file = "Temp/temp.jpg"

    print ("Digite <ESC> para sair / <s> para Salvar")

    while(True):

        ret, frame = camera.read()
        cv2.imshow("Camera", frame)
        k = cv2.waitKey(100)

        if k == 27:
            break

        elif k == ord('s'):
            cv2.imwrite(file,frame)
            break


    cv2.destroyAllWindows()
    camera.release()


def Face():
    camera = 0
    camera = cv2.VideoCapture(camera)

    aux = Usuario[0]
    i=0

    while(True):

        ret, frame = camera.read()
        cv2.imshow("Camera", frame)


        cv2.imwrite("Face/0/"+str(aux)+"_"+str(i)+".jpg",frame)
        print("Mude de posicao")
        cv2.waitKey(2000)
        i+=1

        if i == 11:
            break


    cv2.destroyAllWindows()
    camera.release()
    
def preenchendo_valores():
    with open('Secrets/Usuario.txt','r') as arquivoU:
        for valor in arquivoU:
            Usuario.append(valor)
            
    with open('Secrets/Senha.txt','r') as arquivoS:
        for valor in arquivoS:
            Senha.append(valor)

def cadastro():
    with open('Secrets/Usuario.txt','w') as arquivoU:
        usuario = str(input("Digite seu nome: "))
        arquivoU.write(str(usuario))
        
    with open('Secrets/Senha.txt','w') as arquivoS:
        senha = str(input("Digite sua senha: "))
        arquivoS.write(str(senha))
        

def inicializacao():    
    with open('Secrets/Usuario.txt') as arquivo:
        arquivo.seek(0)
        first_char = arquivo.read(1)
        if not first_char:
            cadastro()
            preenchendo_valores()
            Face()
            main()
        else:
            arquivo.seek(0)
            preenchendo_valores()
            temp()
            validacao()
            
def menu():
    while True:
        print("="*50)
        print("\t\tMenu do Editor")
        print("\nEscolha sua opção:")
        print("\n\n[1] - Editor de fotos")
        print("[0] - Sair\n")

        print("="*50)
    
        aux = int(input("Digite sua opção: "))

        if aux == 0:
            sys.exit()
        
        elif aux == 1:
            edit()
            

        else:
            print("\nOpcao invalida!\n\nTente novamente...\n")
            time.sleep(2)

    
def edit():
    print("="*50)
    print("\t\tEditor de fotos")
    opt = str(input("\n\nDigite o Path: "))
    image = cv2.imread(opt)
    
    print("\n")
    print("="*50)
    
    print("\t\tFoto Cinza\n")
    imagecinza = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    imagemCinza = cv2.cvtColor(imagecinza, cv2.IMREAD_GRAYSCALE)
    plt.imshow(imagemCinza)
    plt.show()
    
    print("\n")
    print("="*50)
    
    print("\t\tFoto Negativa\n")
    img_neg = -1 * image + 255
    plt.figure(figsize=(5,5))
    plt.imshow(img_neg, cmap = "gray")
    plt.show()
    
    print("\n")
    print("="*50)
    
    print("\t\tHistograma\n")
    hist = cv2.calcHist([image],[0],None,[256],[0,256])
    valorintenso = np.array([x for x in range(hist.shape[0])])
    plt.bar(valorintenso, hist[:,0], width = 5)
    plt.show()
    
def main():
    inicializacao()
    menu()
    
main()


Digite <ESC> para sair / <s> para Salvar
face detected [[300 199  89  89]]
img_path Face/0\Giovanni_0.jpg
id: 0
img_path Face/0\Giovanni_1.jpg
id: 0
img_path Face/0\Giovanni_10.jpg
id: 0
img_path Face/0\Giovanni_2.jpg
id: 0
img_path Face/0\Giovanni_3.jpg
id: 0
img_path Face/0\Giovanni_4.jpg
id: 0
img_path Face/0\Giovanni_5.jpg
id: 0
img_path Face/0\Giovanni_6.jpg
id: 0
img_path Face/0\Giovanni_7.jpg
id: 0
img_path Face/0\Giovanni_8.jpg
id: 0
img_path Face/0\Giovanni_9.jpg
id: 0
Confidence 113.2483169041267
Label 0



Rosto não identificado, deseja fazer a validação por usuario e senha? ('S' para sim): 0


SystemExit: 